In [9]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-16 20:23:35--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-16 20:23:35 (71.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [10]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [13]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
llm('write that this is a test')

' This message serves as a placeholder, indicating to the recipient or reader that they are participating in a testing environment and should not interpret any content within it—especially if such environments typically include mock data for automated systems. However, since there\'s no specific context provided about what "this" refers to (a document, an email, software functionality), I will demonstrate the message as though we are instructing someone in a scripted scenario where this is appropriate:\n\n```plaintextANSI\nWrite that THIS IS A TEST. The user interface may present information or content dynamically based on test scenarios designed for quality assurance purposes. Ensure to recognize and disregard any output displayed as part of standard testing procedures, recognizing it might simulate real-time data processing within the contextual application domain being examined (e.g., an e-commerce platform during checkout process simulations).\n```'

In [16]:
print(_)

 This message serves as a placeholder, indicating to the recipient or reader that they are participating in a testing environment and should not interpret any content within it—especially if such environments typically include mock data for automated systems. However, since there's no specific context provided about what "this" refers to (a document, an email, software functionality), I will demonstrate the message as though we are instructing someone in a scripted scenario where this is appropriate:

```plaintextANSI
Write that THIS IS A TEST. The user interface may present information or content dynamically based on test scenarios designed for quality assurance purposes. Ensure to recognize and disregard any output displayed as part of standard testing procedures, recognizing it might simulate real-time data processing within the contextual application domain being examined (e.g., an e-commerce platform during checkout process simulations).
```
